In [1]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader

In [2]:
# Defino una abstraccion para cada tipo de informacion que quiero extraer
# Cada una tiene sus propias propiedades diferentes

In [3]:
class Articulo(Item):
    titulo = Field()
    contenido = Field()

In [4]:
class Review(Item):
    titulo = Field()
    calificacion = Field()

In [5]:
class Video(Item):
    titulo = Field()
    fecha_de_publicacion = Field()

In [ ]:
# CLASE CORE - Al querer hacer extraccion de multiples paginas, heredamos de CrawlSpider
class TripAdvisor(CrawlSpider):
    name = 'hotelestripadvisor'
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36',
        # Como ordenar las columnas en el CSV?
        'FEED_EXPORT_FIELDS': ['id', 'descripcion', 'titular'],
        'CONCURRENT_REQUESTS': 1,  # numero de requerimientos concurrentes
        # Numero maximo de paginas en las cuales voy a descargar items. Scrapy se cierra cuando alcanza este numero
        'CLOSESPIDER_PAGECOUNT': 20,
        'FEED_EXPORT_ENCODING': 'utf-8',  # Tipo de codificacion del archivo de salida
        # En caso de utilizar algun crawler en la nuve como crawlera
        'DOWNLOADER_MIDDLEWARES': {'scrapy_crawlera.CrawleraMiddleware': 610},
        'CRAWLERA_ENABLED': True,
        'CRAWLERA_APIKEY': 'INGRESA_TU_API_KEY',
    }

    # Reduce el espectro de busqueda de URLs. No nos podemos salir de los dominios de esta lista
    # Utilizamos 2 dominios permitidos, ya que los articulos utilizan un dominio diferente
    allowed_domains = ['articulo.mercadolibre.com.ec',
                       'listado.mercadolibre.com.ec']

    # Url semilla a la cual se hara el primer requerimiento
    start_urls = [
        'https://www.tripadvisor.com/Hotels-g303845-Guayaquil_Guayas_Province-Hotels.html']

    # Tiempo de espera entre cada requerimiento. Nos ayuda a proteger nuestra IP.
    # No va a ser dos, va a ser 0.5 * download_delay hasta 1.5 * download delay
    # es decir, va a ser entre 1 y 3 segundos de una manera randomica. Ya es un comportamiento por defecto
    download_delay = 2

    # Tupla de reglas para direccionar el movimiento de nuestro Crawler a traves de las paginas
    rules = (
        Rule(  # Regla de movimiento VERTICAL hacia el detalle de los hoteles
            LinkExtractor(
                allow=r'/Hotel_Review-'  # Si la URL contiene este patron, haz un requerimiento a esa URL
            ), follow=True, callback="parse_hotel"),  # El callback es el nombre de la funcion que se va a llamar con la respuesta al requerimiento hacia estas URLs y extraer datos
    )

    # Funcion a utilizar con MapCompose para realizar limpieza de datos
    def quitarDolar(self, texto):
        return texto.replace("$", "")

    # EL RESPONSE ES EL DE LA URL SEMILLA
    rules = (
        Rule(
            LinkExtractor(
                allow=r'type='
            ), follow=True),  # HORIZONTALIDAD POR TIPO => No tiene callback ya que aqui no voy a extraer datos
        Rule(LinkExtractor(
            allow=r'&page=\d+'
        ), follow=True),  # HORIZONTALIDAD DE PAGINACION EN CADA TIPO => No tiene callback ya que aqui no voy a extraer datos

        # Una regla por cada tipo de contenido donde ire verticalmente
        # Cada una tiene su propia funcion parse que extraera los items dependiendo de la estructura del HTML donde esta cada tipo de item
        Rule(
            LinkExtractor(  # VERTICALIDAD DE REVIEWS
                allow=r'/review/'
            ), follow=True, callback='parse_review'),
        Rule(
            LinkExtractor(  # VERTICALIDAD DE VIDEOS
                allow=r'/video/'
            ), follow=True, callback='parse_video'),
        Rule(
            LinkExtractor(
                allow=r'/news/'  # VERTICALIDAD DE ARTICULOS
            ), follow=True, callback='parse_news'),
        Rule(
            LinkExtractor(
                allow=r'start=',
                # Si los lincks se encuentran en tagas distintos al "a"
                tags=('a', 'button'),
                # Si los links anteriores no llevan un "href", se coloca el que lleva (ej: data-url)
                attrs=('href', 'data-url')
            ), follow=True, callback="parse_farmacia"),
    )

    # DEFINICION DE CADA FUNCION PARSEADORA DE CADA TIPO DE INFORMACION

    # REVIEW
    def parse_review(self, response):
        item = ItemLoader(Review(), response)
        item.add_xpath('titulo', '//h1/text()')
        item.add_xpath(
            'calificacion', '//span[@class="side-wrapper side-wrapper hexagon-content"]/text()')
        yield item.load_item()

    # VIDEO
    def parse_video(self, response):
        item = ItemLoader(Video(), response)
        item.add_xpath('titulo', '//h1/text()')
        item.add_xpath('fecha_de_publicacion',
                       '//span[@class="publish-date"]/text()')
        yield item.load_item()

    # ARTICULO
    def parse_news(self, response):
        item = ItemLoader(Articulo(), response)
        item.add_xpath('titulo', '//h1/text()')
        item.add_xpath('contenido', '//div[@id="id_text"]//*/text()')
        yield item.load_item()

    def parse(self, response):
        # Selectores: Clase de scrapy para extraer datos
        sel = Selector(response)
        # El selector se utiliza cuando se quiere tomar un tag html padre y formar una lista cons us tags hijos
        preguntas = sel.xpath(
            '//div[@id="questions"]//div[@class="question-summary"]')
        i = 0
        for pregunta in preguntas:
            # Instancio mi ITEM con el selector en donde estan los datos para llenarlo
            item = ItemLoader(Pregunta(), pregunta)

            # Lleno las propiedades de mi ITEM a traves de expresiones XPATH a buscar dentro del selector "pregunta"
            item.add_xpath('pregunta', './/h3/a/text()')
            item.add_xpath('descripcion', './/div[@class="excerpt"]/text()')
            item.add_value('id', i)
            i += 1
            # Hago Yield de la informacion para que se escriban los datos en el archivo
            yield item.load_item()


In [ ]:
# EJECUCION

In [ ]:
# Por terminal: 
# scrapy runspider 4_eluniverso.py -o resultados.csv

In [ ]:
#  CORRIENDO SCRAPY SIN LA TERMINAL

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'datos_de_salida.json'
})
process.crawl(ElUniversoSpider)
process.start()